<a href="https://colab.research.google.com/github/kwanglo/mge51101-20195171/blob/master/final_project/multi_sentiment_CNN_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
!nvidia-smi

Tue Jun  2 03:44:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
!pip3 install konlpy
!pip3 install soynlp

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.6MB 34.6MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
     |████████████████████████████████| 419kB 4.9MB/s 


In [0]:
import os
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Twitter

import pandas as pd
import numpy as np

In [0]:
path='/gdrive/My Drive/Colab Notebooks/Final Project/dataset/'
df = pd.read_excel(path+"한국어_단발성_대화_데이터셋.xlsx")

In [0]:
df = df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4','공포',5468])

In [8]:
df.groupby("Emotion").count()

,Sentence
Emotion,
공포,5468
놀람,5898
분노,5665
슬픔,5267
중립,4830
행복,6037
혐오,5429


In [9]:
df

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


# Split dataset

In [0]:
# Char to int
df.loc[(df.Emotion == "중립"),"Emotion"] = 0
df.loc[(df.Emotion == "공포"),"Emotion"] = 1
df.loc[(df.Emotion == "놀람"),"Emotion"] = 2
df.loc[(df.Emotion == "분노"),"Emotion"] = 3
df.loc[(df.Emotion == "슬픔"),"Emotion"] = 4
df.loc[(df.Emotion == "행복"),"Emotion"] = 5
df.loc[(df.Emotion == "혐오"),"Emotion"] = 6

In [11]:
df.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,1
1,그냥 내 느낌일뿐겠지?,1
2,아직너무초기라서 그런거죠?,1
3,유치원버스 사고 낫다던데,1
4,근데 원래이런거맞나요,1


In [0]:
data = df

In [0]:
# 숫자로 이미지 나눌때 
#중립
Neutral = data[data["Emotion"] == 0]
#공포
Fear = data[data["Emotion"] == 1]
Fear.head()
#놀람
Surprise = data[data["Emotion"] == 2]
#분노
Anger = data[data["Emotion"] == 3]
#슬픔
Sad = data[data["Emotion"] == 4]
#행복
Happy = data[data["Emotion"] == 5]
#혐오
Disgust = data[data["Emotion"] == 6]

In [0]:
rnd_num = 2020
Fear_train = Fear.sample(frac=0.7, random_state=rnd_num)
Fear_test = Fear.drop(Fear_train.index)

Surprise_train = Surprise.sample(frac=0.7, random_state=rnd_num)
Surprise_test = Surprise.drop(Surprise_train.index)

Anger_train = Anger.sample(frac=0.7, random_state=rnd_num)
Anger_test = Anger.drop(Anger_train.index)

Sad_train = Sad.sample(frac=0.7, random_state=rnd_num)
Sad_test = Sad.drop(Sad_train.index)

Neutral_train = Neutral.sample(frac=0.7, random_state=rnd_num)
Neutral_test = Neutral.drop(Neutral_train.index)

Happy_train = Happy.sample(frac=0.7, random_state=rnd_num)
Happy_test = Happy.drop(Happy_train.index)

Disgust_train = Disgust.sample(frac=0.7, random_state=rnd_num)
Disgust_test = Disgust.drop(Disgust_train.index)

train = pd.concat([Fear_train,Surprise_train,Anger_train,Sad_train,Neutral_train,Happy_train,Disgust_train])
test = pd.concat([Fear_test,Surprise_test,Anger_test,Sad_test,Neutral_test,Happy_test,Disgust_test])

In [15]:
train.groupby("Emotion").count()

,Sentence
Emotion,
0,3381
1,3828
2,4129
3,3965
4,3687
5,4226
6,3800


In [16]:
test.groupby("Emotion").count()

,Sentence
Emotion,
0,1449
1,1640
2,1769
3,1700
4,1580
5,1811
6,1629


# Preprocssing data

In [0]:
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def tokenizer(text): # create a tokenizer function
    okt = Okt()
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    x = okt.morphs(text , stem= True)
    return x

In [0]:
stop_words_set = pd.read_csv(path+'stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words)

In [0]:
import torch
from torchtext import data
from torchtext import datasets
from soynlp.tokenizer import MaxScoreTokenizer
SEED = 3432

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(tokenize=tokenizer, stop_words = stop_words)
LABEL = data.LabelField()

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train['Sentence'],
                                                                                    train['Emotion'], 
                                                                                    random_state=2020, 
                                                                                    test_size=0.3)

In [0]:
train_set = pd.concat([train_inputs, train_labels], axis=1)
valid_set = pd.concat([validation_inputs, validation_labels], axis=1)
test_set = test

In [23]:
test_set.groupby("Emotion").count()

,Sentence
Emotion,
0,1449
1,1640
2,1769
3,1700
4,1580
5,1811
6,1629


In [0]:
train_set.to_csv(path+'sentiment_train.csv',index=False, encoding='utf-8')
valid_set.to_csv(path+'sentiment_valid.csv',index=False, encoding='utf-8')
test_set.to_csv(path+'sentiment_test.csv',index=False, encoding='utf-8')

In [0]:
from torchtext.data import TabularDataset
fields = [("Sentence", TEXT),("Emotion", LABEL)]

In [0]:
train_data,valid_data, test_data = data.TabularDataset.splits(
                                        path = path,
                                        train = 'sentiment_train.csv',
                                        validation = 'sentiment_valid.csv',
                                        test = 'sentiment_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [27]:
vars(train_data[3])

{'Emotion': '5', 'Sentence': ['어제', '런닝맨', '완전', '재밌다']}

In [28]:
train_set.head()

,Sentence,Emotion
15518,대본 없으면 말도 어버버하는 AI걸린닭,3
16824,내거 믿는 진실은 결국은 거짓선동인가?,3
21694,그러다가 제대하면 헤어지고..이런 여자들은 그냥 휴가 나올 때 한번 하고이런 용도라...,4
29253,어제 런닝맨 완전 재밌었어요!,5
8714,근데 딱 두줄이...,2


In [0]:
import torchtext
vec = torchtext.vocab.Vectors('wiki.ko.vec', cache=path)

In [0]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = vec, 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [31]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f128bcb6158>, {'2': 0, '5': 1, '3': 2, '1': 3, '6': 4, '4': 5, '0': 6})


In [0]:
from torchtext.data import Iterator, BucketIterator
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = BucketIterator.splits(
 (train_data, valid_data, ), 
 batch_sizes=(BATCH_SIZE, BATCH_SIZE),
 device=device,
 sort_key=lambda x: len(x.Sentence), 
 sort_within_batch=False, sort=False,
 repeat=False
)

test_iterator  = Iterator(test_data, batch_size=BATCH_SIZE, device= device, sort_key=lambda x: len(x.Sentence), sort_within_batch=False, sort=False, repeat=False)

In [0]:
from torchtext.data import Iterator, BucketIterator
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort = False)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        text = text.permute(1, 0)        
        embedded = self.embedding(text)

        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [36]:
len(LABEL.vocab)

7

In [37]:
TEXT.vocab.stoi[TEXT.pad_token]

1

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,058,307 trainable parameters


In [39]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1452,  0.3633, -0.3033,  ..., -0.1686,  0.0110, -0.0240],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.Sentence)
        
        loss = criterion(predictions, batch.Emotion)
        
        acc = categorical_accuracy(predictions, batch.Emotion)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.Sentence)
            
            loss = criterion(predictions, batch.Emotion)
            
            acc = categorical_accuracy(predictions, batch.Emotion)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [46]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 1.590 | Train Acc: 37.17%
	 Val. Loss: 1.422 |  Val. Acc: 44.93%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 1.269 | Train Acc: 51.42%
	 Val. Loss: 1.396 |  Val. Acc: 46.91%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 1.000 | Train Acc: 62.61%
	 Val. Loss: 1.488 |  Val. Acc: 46.07%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.749 | Train Acc: 73.07%
	 Val. Loss: 1.658 |  Val. Acc: 45.25%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.533 | Train Acc: 81.11%
	 Val. Loss: 1.944 |  Val. Acc: 44.24%
Epoch: 06 | Epoch Time: 0m 10s
	Train Loss: 0.389 | Train Acc: 86.34%
	 Val. Loss: 2.244 |  Val. Acc: 43.19%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.302 | Train Acc: 89.30%
	 Val. Loss: 2.515 |  Val. Acc: 42.60%
Epoch: 08 | Epoch Time: 0m 10s
	Train Loss: 0.238 | Train Acc: 91.49%
	 Val. Loss: 2.787 |  Val. Acc: 42.71%
Epoch: 09 | Epoch Time: 0m 10s
	Train Loss: 0.193 | Train Acc: 93.23%
	 Val. Loss: 3.087 |  Val. Acc: 42.25%
Epoch: 10 | Epoch T

In [47]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.386 | Test Acc: 45.87%


In [48]:
tokenizer("오늘 되게 우울하다")

['오늘', '되게', '우울하다']

In [0]:
def predict_class(model, sentence, min_len = 4):
    model.eval()
    # 이 부분에서 그냥 바로 tokenizing
    tokenized = tokenizer(sentence)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [0]:
def emotion_classifier(logits):
  global sentiment
  if logits == "0":
    sentiment = '중립'
  elif logits == "1":
    sentiment = '공포'
  elif logits == "2":
    sentiment = '놀람'
  elif logits == "3":
    sentiment = '분노'
  elif logits == "4":
    sentiment = '슬픔'
  elif logits == "5":
    sentiment = '행복'
  elif logits == "6":
    sentiment = '혐오'

  return sentiment

In [56]:
pred_sentence = input()
pred_class = predict_class(model, pred_sentence)
logit = LABEL.vocab.itos[pred_class]

print(f'이 문장의 감정은: {emotion_classifier(logit)}')

그래도 어찌어찌 끝내서 다행이다
이 문장의 감정은: 행복
